In [ ]:
from google.colab import drive
drive.mount('/gdrive')

In [ ]:
!pip install pretty_midi
!cp '/gdrive/MyDrive/Classical_Music_Transformer_Integration/data/pretrained_model/classical_music_transformer/inference/model.py' ./model.py
!cp '/gdrive/MyDrive/Classical_Music_Transformer_Integration/data/pretrained_model/classical_music_transformer/inference/settings.py' ./settings.py
!cp '/gdrive/MyDrive/Classical_Music_Transformer_Integration/data/pretrained_model/classical_music_transformer/inference/util.py' ./util.py

In [ ]:
from model import *
from util import *
from settings import *

In [ ]:
### init
d_type = 5
d_barpos = 34
d_tone = 25
d_chord = 109
d_dur = 65
d_pitch = 129
d_attention = 512
heads = 8
N = 12
model = Transformer()
if torch.cuda.is_available():
	model.cuda()

model.load_state_dict(torch.load("/gdrive/MyDrive/Classical_Music_Transformer_Integration/data/checkpoint/4.pt"))

<All keys matched successfully>

In [ ]:
### make start seq
start_seq = ["bar", "phrase", "pos0", "noteC3dur8"]

### seq to input
type_oh, barpos_oh, tone_oh, chord_oh, dur_oh, pitch_oh = seq2input(start_seq)

In [ ]:
### random sampling experiment switch
rses = True

In [ ]:
def nucleus_sampling(vec2, ro = 0.9):
    vec = np.array(vec2)
    sorted_vec = np.sort(vec)[::-1]
    sorted_index = np.argsort(vec)[::-1]
    cumsum_sorted_vec = np.cumsum(sorted_vec)
    dropped = cumsum_sorted_vec > ro
    if np.sum(dropped) > 0:
        last_index = np.where(dropped)[0][0] + 1
        candi_index = sorted_index[:last_index]
    else:
        candi_index = sorted_index[:]
        
    candi_probs = [vec[i] for i in candi_index]
    candi_probs /= sum(candi_probs)

    if rses:

        try:
            candi_probs_sorted = np.sort(candi_probs)
            max_1 = candi_probs_sorted[-1]
            max_2 = candi_probs_sorted[-2]
            mean = (max_1 + max_2) / 2

            idx_1 = np.where(candi_probs == max_1)
            idx_2 = np.where(candi_probs == max_2)

            candi_probs[idx_1] = mean
            candi_probs[idx_2] = mean
        except:
            pass
    
    choice = np.random.choice(candi_index, size=1, p=candi_probs)[0] 

    return choice

In [ ]:
### sample how many events
num_event = 1000

### sampling
for i in range(len(start_seq) - 1, num_event):
	if i > 127:
		type_input = type_oh[i-127:i+1].unsqueeze(0)
		barpos_input = barpos_oh[i-127:i+1].unsqueeze(0)
		tone_input = tone_oh[i-127:i+1].unsqueeze(0)
		chord_input = chord_oh[i-127:i+1].unsqueeze(0)
		dur_input = dur_oh[i-127:i+1].unsqueeze(0)
		pitch_input = pitch_oh[i-127:i+1].unsqueeze(0)
	else:
		type_input = type_oh[:i+1].unsqueeze(0)
		barpos_input = barpos_oh[:i+1].unsqueeze(0)
		tone_input = tone_oh[:i+1].unsqueeze(0)
		chord_input = chord_oh[:i+1].unsqueeze(0)
		dur_input = dur_oh[:i+1].unsqueeze(0)
		pitch_input = pitch_oh[:i+1].unsqueeze(0)

	with torch.no_grad():
		h, pred_type = model.forward_hidden(type_input, barpos_input, tone_input, chord_input, dur_input, pitch_input, create_mask(type_input))

	pred_type = choose_type(pred_type)
	with torch.no_grad():
		pred_barpos, pred_tone, pred_chord, pred_dur, pred_pitch = model.forward_sampling(h, pred_type)

	type_vec = torch.tensor([0]*d_type).type(torch.FloatTensor).cuda()
	barpos_vec = torch.tensor([0]*d_barpos).type(torch.FloatTensor).cuda()
	tone_vec = torch.tensor([0]*d_tone).type(torch.FloatTensor).cuda()
	chord_vec = torch.tensor([0]*d_chord).type(torch.FloatTensor).cuda()
	dur_vec = torch.tensor([0]*d_dur).type(torch.FloatTensor).cuda()
	pitch_vec = torch.tensor([0]*d_pitch).type(torch.FloatTensor).cuda()

	### Bar & Pos
	if pred_type[0, -1, 0] == 1:
		type_vec[0] = 1

		# print('Bar & Pos')
		# print(pred_barpos)

		sampled_barpos = nucleus_sampling(pred_barpos[0, -1, :].softmax(-1).tolist())
		barpos_vec[sampled_barpos] = 1

		tone_vec[0] = 1
		chord_vec[0] = 1
		dur_vec[0] = 1
		pitch_vec[0] = 1

	### tone
	elif pred_type[0, -1, 2] == 1:
		type_vec[2] = 1
		barpos_vec[0] = 1

		# print('Tone')
		# print(pred_tone)

		sampledtone = nucleus_sampling(pred_tone[0, -1, 1:].softmax(-1).tolist())
		curtone = sampledtone
		tone_vec[sampledtone + 1] = 1
		chord_vec[0] = 1
		dur_vec[0] = 1
		pitch_vec[0] = 1

	### chord
	elif pred_type[0, -1, 3] == 1:
		type_vec[3] = 1
		barpos_vec[0] = 1
		tone_vec[0] = 1

		# print('Chord')
		# print(pred_chord)

		sampledchord = nucleus_sampling(pred_chord[0, -1, 1:].softmax(-1).tolist())
		curchord = sampledchord
		chord_vec[sampledchord + 1] = 1
		dur_vec[0] = 1
		pitch_vec[0] = 1

	### phrase
	elif pred_type[0, -1, 4] == 1:
		type_vec[4] = 1
		barpos_vec[0] = 1
		tone_vec[0] = 1
		chord_vec[0] = 1
		dur_vec[0] = 1
		pitch_vec[0] = 1

	### notes
	else:
		type_vec[1] = 1
		barpos_vec[0] = 1
		tone_vec[0] = 1
		chord_vec[0] = 1

		# print('Notes Dur')
		# print(pred_dur)
  
		sampleddur = nucleus_sampling(pred_dur[0, -1, 1:].softmax(-1).tolist())
		dur_vec[sampleddur + 1] = 1

		# print('Notes Pitch')
		# print(pred_pitch)
  
		sampledpitch = nucleus_sampling(pred_pitch[0, -1, 1:].softmax(-1).tolist())
		pitch_vec[sampledpitch + 1] = 1

	type_oh = torch.cat([type_oh, type_vec.unsqueeze(0)], 0)
	barpos_oh = torch.cat([barpos_oh, barpos_vec.unsqueeze(0)], 0)
	tone_oh = torch.cat([tone_oh, tone_vec.unsqueeze(0)], 0)
	chord_oh = torch.cat([chord_oh, chord_vec.unsqueeze(0)], 0)
	dur_oh = torch.cat([dur_oh, dur_vec.unsqueeze(0)], 0)
	pitch_oh = torch.cat([pitch_oh, pitch_vec.unsqueeze(0)], 0)

In [ ]:
### write to midi
item2midi(onehot2item(type_oh, barpos_oh, tone_oh, chord_oh, dur_oh, pitch_oh), "output01.mid")

In [ ]:
### write to midi
item2midi(onehot2item(type_oh, barpos_oh, tone_oh, chord_oh, dur_oh, pitch_oh), "output02.mid")

In [ ]:
### write to midi
item2midi(onehot2item(type_oh, barpos_oh, tone_oh, chord_oh, dur_oh, pitch_oh), "output03.mid")

In [ ]:
### write to midi
item2midi(onehot2item(type_oh, barpos_oh, tone_oh, chord_oh, dur_oh, pitch_oh), "output04.mid")

In [ ]:
### write to midi
item2midi(onehot2item(type_oh, barpos_oh, tone_oh, chord_oh, dur_oh, pitch_oh), "output05.mid")

In [ ]:
### write to midi
item2midi(onehot2item(type_oh, barpos_oh, tone_oh, chord_oh, dur_oh, pitch_oh), "output06.mid")

In [ ]:
### write to midi
item2midi(onehot2item(type_oh, barpos_oh, tone_oh, chord_oh, dur_oh, pitch_oh), "output07.mid")

In [ ]:
### write to midi
item2midi(onehot2item(type_oh, barpos_oh, tone_oh, chord_oh, dur_oh, pitch_oh), "output08.mid")

In [ ]:
### write to midi
item2midi(onehot2item(type_oh, barpos_oh, tone_oh, chord_oh, dur_oh, pitch_oh), "output09.mid")

In [ ]:
### write to midi
item2midi(onehot2item(type_oh, barpos_oh, tone_oh, chord_oh, dur_oh, pitch_oh), "output10.mid")

In [ ]:
### write to midi
item2midi(onehot2item(type_oh, barpos_oh, tone_oh, chord_oh, dur_oh, pitch_oh), "output11.mid")

In [ ]:
### write to midi
item2midi(onehot2item(type_oh, barpos_oh, tone_oh, chord_oh, dur_oh, pitch_oh), "output12.mid")

In [ ]:
### write to midi
item2midi(onehot2item(type_oh, barpos_oh, tone_oh, chord_oh, dur_oh, pitch_oh), "output13.mid")